In [1]:
#Name: Arundarasi Rajendran
#Student ID: 100946579

In [2]:
#A library for parsing HTML and XML documents. It creates parse trees from page source codes and allows easy extraction of data.
from bs4 import BeautifulSoup
# requests: A library for making HTTP requests. It is used to fetch the content of web pages.
import requests
# pandas: A data manipulation and analysis library that provides data structures like DataFrames to handle and analyze data in tabular form.
import pandas as pd
# numpy: A library for numerical operations in Python. It is used here to handle NaN values and perform data cleaning operations.
import numpy as np

In [3]:
#Referred from https://nanonets.com/blog/web-scraping-with-python-tutorial/

# Function to extract Product Title
def get_title(soup):
    """
    Extracts the product title from the BeautifulSoup object.
    """
    try:
        # Locate the title element using its id
        title = soup.find("span", attrs={"id":'productTitle'})

        # If title element is found, get its text and clean it
        if title:
            title_value = title.text
            #strip function removes empty spaces in the value
            title_string = title_value.strip()
        else:
            title_string = ""

    except AttributeError:
        # If there's an error finding the title element, return an empty string
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):
    """
    Extracts the product price from the BeautifulSoup object.
    """
    try:
        # Attempt to locate the 'ourprice' element and get its text
        price = soup.find("span", attrs={'id':'priceblock_ourprice'})
        if price:
            #strip function removes empty spaces in the value
            price = price.string.strip()
        else:
            # If 'ourprice' is not found, try to locate the 'dealprice' element
            price = soup.find("span", attrs={'id':'priceblock_dealprice'})
            if price:
            #strip function removes empty spaces in the value
                price = price.string.strip()
            else:
                # If neither price is found, return an empty string
                price = ""

    except AttributeError:
        # If there's an error locating price elements, return an empty string
        price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):
    """
    Extracts the product rating from the BeautifulSoup object.
    """
    try:
        # Attempt to locate the rating element with a specific class
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'})
        if rating:
            #strip function removes empty spaces in the value
            rating = rating.string.strip()
        else:
            # If specific rating class is not found, try a general rating element
            rating = soup.find("span", attrs={'class':'a-icon-alt'})
            if rating:
            #strip function removes empty spaces in the value
                rating = rating.string.strip()
            else:
                # If rating is not found, return an empty string
                rating = ""

    except AttributeError:
        # If there's an error locating rating elements, return an empty string
        rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    """
    Extracts the number of user reviews from the BeautifulSoup object.
    """
    try:
        # Locate the review count element using its id and get its text
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'})
        if review_count:
            #strip function removes empty spaces in the value
            review_count = review_count.string.strip()
        else:
            # If review count is not found, return an empty string
            review_count = ""

    except AttributeError:
        # If there's an error locating review count element, return an empty string
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    """
    Extracts the availability status of the product from the BeautifulSoup object.
    """
    try:
        # Locate the availability element using its id
        available = soup.find("div", attrs={'id':'availability'})
        if available:
            available = available.find("span")
            if available:
            #strip function removes empty spaces in the value
                available = available.string.strip()
            else:
                available = "Not Available"
        else:
            available = "Not Available"

    except AttributeError:
        # If there's an error locating availability element, return a default value
        available = "Not Available"

    return available

In [8]:

if __name__ == '__main__':
    """
    Main block to execute the web scraping and data extraction.
    """
    # Define the headers to mimic a browser request
    #Got my useragent from https://explore.whatismybrowser.com/useragents/parse/?analyse-my-user-agent=yes#google_vignette or it works with useragent empty as well
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:128.0) Gecko/20100101 Firefox/128.0', 'Accept-Language': 'en-US, en;q=0.5'})

    # URL of the webpage to scrape
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # Send an HTTP GET request to the URL
    webpage = requests.get(URL, headers=HEADERS)

    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Find all product links on the page
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the extracted links in a list
    links_list = []

    # Extract the href attribute from each link and add to the list
    for link in links:
        href = link.get('href')
        if href:
            links_list.append(href)

    # Initialize a dictionary to store the extracted product details
    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop through each product link to extract detailed information
    for link in links_list:
        # Fetch the product page content using the full URL
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        # Create a BeautifulSoup object for the product page
        #Referred from https://realpython.com/beautiful-soup-web-scraper-python/
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Call functions to extract product details and append to dictionary
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    # Create a DataFrame from the dictionary
    amazon_df = pd.DataFrame.from_dict(d)

    # Replace empty strings in the 'title' column with NaN and drop rows with NaN titles
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])

    # Save the DataFrame to a CSV file
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [9]:
#display amazon dataframe
amazon_df

,title,price,rating,reviews,availability
1,PlayStation 4 500GB Console (Renewed),,,"1,221 ratings",In Stock
2,PlayStation 4 Slim 1TB Console - Black (Renewed),,,"1,638 ratings",
3,Sony PlayStation 4 Slim Limited Edition 1TB Ga...,,,"1,443 ratings",In stock
4,PlayStation®4 Console – Call of Duty® Modern W...,,,91 ratings,In Stock
5,"Sony PlayStation 4 Pro w/ Accessories, 1TB HDD...",,,971 ratings,Only 11 left in stock - order soon.
6,Sony Playstation PS4 1TB Black Console,,,"1,509 ratings",
7,PlayStation 4 Slim 1TB Console,,,"15,726 ratings",In Stock
8,PlayStation 4 Slim 500GB Console [Discontinued...,,4.5 out of 5 stars,"1,405 ratings",In Stock
10,"Playstation SONY 4, 500GB Slim System [CUH-221...",,4.3 out of 5 stars,438 ratings,Not Available
11,Sony PlayStation 4 Pro 1TB Console - Black (PS...,,4.4 out of 5 stars,"4,140 ratings",Not Available
